In [1]:
import sys
sys.path.insert(0, '../../')
import argparse
import numpy as np
import pandas as pd
from scipy.stats import norm

from kernels.nn import ImplicitDenseNetKernel
from model.ick import ICK, AdditiveICK
from utils.train import EnsembleTrainer
from utils.helpers import *

# To make this notebook's output stable across runs
np.random.seed(2020)
torch.manual_seed(2020)
torch.cuda.manual_seed(2020)
torch.cuda.manual_seed_all(2020)
torch.backends.cudnn.deterministic = True

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# 1. Load IHDP data

In [2]:
# Load data repetition function
def load_data_rep(data_train, data_test, i):
    # Training data
    X_train = data_train['x'][:, :, i]
    T_train = data_train['t'][:, i:i + 1]
    Y_train = data_train['yf'][:, i:i + 1]
    mu0_train = data_train['mu0'][:, i:i + 1]  # mu0 is the mean of control outcome
    mu1_train = data_train['mu1'][:, i:i + 1]  # mu1 is the mean of treatment outcome
    
    # Test data
    X_test = data_test['x'][:, :, i]
    T_test = data_test['t'][:, i:i + 1]
    Y0_test = data_test['yf'][:, i:i + 1] * (1.0 - data_test['t'][:, i:i + 1])  # Y0 is the outcome for control
    Y0_test += data_test['ycf'][:, i:i + 1] * data_test['t'][:, i:i + 1]
    Y1_test = data_test['ycf'][:, i:i + 1] * (1.0 - data_test['t'][:, i:i + 1])  # Y1 is the outcome for treatment
    Y1_test += data_test['yf'][:, i:i + 1] * data_test['t'][:, i:i + 1]
    mu0_test = data_test['mu0'][:, i:i + 1]
    mu1_test = data_test['mu1'][:, i:i + 1]
    
    # Log-likelihood
    ll_test = np.mean(np.log(norm.cdf(Y0_test - mu0_test + 0.5) - norm.cdf(Y0_test - mu0_test - 0.5))) + \
    np.mean(np.log(norm.cdf(Y1_test - mu1_test + 0.5) - norm.cdf(Y1_test - mu1_test - 0.5)))
    
    # Return data
    return X_train, T_train, Y_train, mu0_train, mu1_train, X_test, \
    T_test, Y0_test, Y1_test, mu0_test, mu1_test, ll_test

data_train = np.load('../../data/ihdp_npci_1-100.train.npz', allow_pickle=True)
data_test = np.load('../../data/ihdp_npci_1-100.test.npz', allow_pickle=True)

X_train, T_train, Y_train, mu0_train, mu1_train, X_test, T_test, \
Y0_test, Y1_test, mu0_test, mu1_test, ll_test = load_data_rep(
    data_train=data_train,
    data_test=data_test,
    i=0)
mu_test = mu1_test - mu0_test

# Define dataset and dataloaders
train_df = pd.DataFrame(X_train)
train_df["Y"] = Y_train
train_df["T"] = T_train
X0_train = train_df[train_df["T"]==0.0].iloc[:,0:X_train.shape[1]].to_numpy()
Y0_train = train_df[train_df["T"]==0.0]["Y"].to_numpy()
X1_train = train_df[train_df["T"]==1.0].iloc[:,0:X_train.shape[1]].to_numpy()
Y1_train = train_df[train_df["T"]==1.0]["Y"].to_numpy()
data_generators_Y0 = create_generators_from_data(X0_train, Y0_train, X_test, Y0_test)
data_generators_Y1 = create_generators_from_data(X1_train, Y1_train, X_test, Y1_test)

# 2. Define ICK-CMGP model

$Y_0(x) = \alpha_1 f_1(x) + \alpha_2 f_2(x) + 0 \\$
$Y_1(x) = 0 + \alpha_2 f_2(x) + \alpha_3 f_3(x)$

In [3]:
# Y0(x) = alpha1 * f1(x) + alpha2 * f2(x) + 0 * f3(x)
# Y1(x) = 0 * f1(x) + alpha2 * f2(x) + alpha3 * f3(x)
alpha1, alpha2, alpha3 = 0.25, 0.5, 0.25
f1 = ICK(
    kernel_assignment=['ImplicitDenseNetKernel'],
    kernel_params={
        'ImplicitDenseNetKernel':{
            'input_dim': X_train.shape[1],
            'latent_feature_dim': 16,
            'num_blocks': 1, 
            'num_layers_per_block': 1, 
            'num_units': 64
        }
    }
)
f2 = ICK(
    kernel_assignment=['ImplicitDenseNetKernel'],
    kernel_params={
        'ImplicitDenseNetKernel':{
            'input_dim': X_train.shape[1],
            'latent_feature_dim': 16,
            'num_blocks': 2, 
            'num_layers_per_block': 1, 
            'num_units': 64
        }
    }
)
f3 = ICK(
    kernel_assignment=['ImplicitDenseNetKernel'],
    kernel_params={
        'ImplicitDenseNetKernel':{
            'input_dim': X_train.shape[1],
            'latent_feature_dim': 16,
            'num_blocks': 1, 
            'num_layers_per_block': 1, 
            'num_units': 64
        }
    }
)
num_estimators = 100
ensemble_Y0 = [AdditiveICK(
    components=[f1,f2], 
    component_assignment=[[0],[0]], 
    coeffs=[alpha1,alpha2], 
    weighted=[False,False]
) for _ in range(num_estimators)]
ensemble_Y1 = [AdditiveICK(
    components=[f2,f3], 
    component_assignment=[[0],[0]], 
    coeffs=[alpha2,alpha3], 
    weighted=[False,False]
) for _ in range(num_estimators)]

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optim = 'sgd'
optim_params = {
    'lr': 1e-6, 
    'momentum': 0.9,
    'weight_decay': 0
}
epochs, patience = 200, 10
trainer_Y0 = EnsembleTrainer(
    model=ensemble_Y0,
    data_generators=data_generators_Y0,
    optim=optim,
    optim_params=optim_params, 
    model_save_dir=None,
    device=device,
    epochs=epochs,
    patience=patience
)
trainer_Y1 = EnsembleTrainer(
    model=ensemble_Y1,
    data_generators=data_generators_Y1,
    optim=optim,
    optim_params=optim_params, 
    model_save_dir=None,
    device=device,
    epochs=epochs,
    patience=patience
)
trainer_Y0.train()
trainer_Y1.train()

Training started:

Epoch 1/200
Learning rate: 0.000001


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Training time - 1s - loss 6.9504
Validation:
0s - loss 4.2275

Epoch 2/200
Learning rate: 0.000001


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Training time - 1s - loss 3.8035
Validation:
0s - loss 2.0110

Epoch 3/200
Learning rate: 0.000001


...

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Training time - 0s - loss 1.0323
Validation:
0s - loss 1.1608

Epoch 106/200
Learning rate: 0.000001


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Training time - 0s - loss 1.0303
Validation:
0s - loss 1.1610

Early stopping - patience reached
Restoring the best model


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished


In [9]:
mu0_test_pred, std0_test_pred, y0_test_true = trainer_Y0.predict()
mu1_test_pred, std1_test_pred, y1_test_true = trainer_Y1.predict()
mu_test_pred = mu1_test_pred - mu0_test_pred

# PEHE
pehe_test = np.sqrt(np.mean((mu_test_pred - mu_test) ** 2))
print('PEHE:             %.4f' % (pehe_test))

PEHE:             4.4211
